### Select Datasets for comparing RSAST with SAST Ridge, HIVE-COTE, ROCKET, TS-CHIEF

In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [1]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown','BirdChicken','Coffee',  'ArrowHead']
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


NameError: name 'tsc_dataset_names' is not defined

In [9]:
runs = 1
#list
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []

p=100
k=10
for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        print("ROCKET: kernels=10_000")
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        
        print("RSAST: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        start = time.time()
        random_state = None
        rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=k, classifier=RidgeClassifierCV())
        rsast_ridge.fit(X_train, y_train)
        end = time.time()
        list_score.append(rsast_ridge.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        list_rpoint.append(str(p))
        list_nb_per_class.append(str(k))
        list_method.append("Rsast")
        
        

df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method

ds=Chinatown
ROCKET: kernels=10_000
RSAST: n_random_points=100 nb_inst_per_class=10
ROCKET: kernels=10_000
RSAST: n_random_points=100 nb_inst_per_class=10
ROCKET: kernels=10_000
RSAST: n_random_points=100 nb_inst_per_class=10
ds=BirdChicken
ROCKET: kernels=10_000
RSAST: n_random_points=100 nb_inst_per_class=10
ROCKET: kernels=10_000
RSAST: n_random_points=100 nb_inst_per_class=10


In [ ]:
df_result=pd.DataFrame(df_result)
# create a pivot table
df_result.to_csv("df_overall_comparison_results.csv") 
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_overall_comparison.csv") 